In [52]:
!pip install lightgbm
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install catboost
!pip install xgboost
!pip install imblearn
!pip install hyperopt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.6 MB 825.8 kB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.6 MB 1.5 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.6 MB 2.0 MB/s eta 0:00:01
   ----------------- ---------------------- 0.7/1.6 MB 3.3 MB/s eta 0:00:01
   ----------------- ---------------------- 0.7/1.6 MB 3.3 MB/s eta 0:00:01
   ------------------------------- -------- 1.3/1.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/491.3 kB ? eta -:--:--
   ----------------- --------------------- 225.3/491.3 kB 13.4 MB/s eta 0:00:01
   --------------------------- ------------ 337.9/491.3 kB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 491.3/491.3 kB 5.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score,precision_score,recall_score,confusion_matrix, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score

In [4]:
import glob
import os


path = "data_alfa"


all_files = glob.glob(os.path.join(path, "test*.csv"))


data_frames = []


for filename in all_files:
    df = pd.read_csv(filename)
    data_frames.append(df)


merged_df = pd.concat(data_frames, ignore_index=True)


merged_df.to_csv("data_alfa/merged_test.csv", index=False)

print("Объединение завершено. Результат сохранён в 'merged_test.csv'.")



ValueError: No objects to concatenate

In [3]:
file_path = "merged_train.csv"
df = pd.read_csv(file_path)
df.head()
del df["smpl"]

In [4]:
X = df.drop(["target"],axis=1)
y = df["target"]

In [5]:
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}

In [8]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X, y)

In [11]:
# Test data

file_path = "data/test.csv"
df = pd.read_csv(file_path)
df.head()

scaler = StandardScaler()
scaler.fit(df)
output_X = scaler.transform(df)

output_df = pd.read_csv('data/input/baseline_submission_case1.csv', index_col='id')

FileNotFoundError: [Errno 2] No such file or directory: 'data/test.csv'

## BaseLine модель (LightGbm)

In [12]:
# rec 32
# prec 94
# roc-auc 98

import lightgbm as lgb

params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 31,
    'max_depth': 10,
    'min_child_samples': 10,
    'class_weight': 'balanced',
    'n_estimators': 1000
}

lgb_clf = lgb.LGBMClassifier(**params)
lgb_clf.fit(X_resampled,y_resampled)

lgb_clf.predict(X_test)
y_pred = lgb_clf.predict(X_test)
y_pred_proba = lgb_clf.predict_proba(X_test)[:, 1]

print("recall:", recall_score(y_test,y_pred))
print("precision:", precision_score(y_test,y_pred))
print("roc-auc:", roc_auc_score(y_test,y_pred_proba))
print("confusion matrix:", confusion_matrix(y_test,y_pred))


[LightGBM] [Info] Number of positive: 37888, number of negative: 37888
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.129042 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 106732
[LightGBM] [Info] Number of data points in the train set: 75776, number of used features: 419
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
recall: 0.30113636363636365
precision: 0.9814814814814815
roc-auc: 0.9828809992675399
confusion matrix: [[7570    1]
 [ 123   53]]


In [13]:
threshold = 0.3
y_pred_custom = (y_pred_proba >= threshold).astype(int)

precision = precision_score(y_test, y_pred_custom)
recall = recall_score(y_test, y_pred_custom)
print(f"Precision: {precision}, Recall: {recall}")

Precision: 0.7364341085271318, Recall: 0.5397727272727273


In [14]:
output = lgb_clf.predict_proba(output_X)[:,1]
output_df['target'] = output
output_df.to_csv('data/output/submission.csv', index=True)

NameError: name 'output_X' is not defined

## Оптимизация с HyperOpt


In [15]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
import lightgbm as lgb
from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt.fmin import fmin
import pandas as pd
from sklearn.metrics import recall_score, precision_score, roc_auc_score, confusion_matrix

# Определение пространства поиска гиперпараметров
space = {
    'num_leaves': hp.quniform('num_leaves', 20, 100, 1),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'min_child_samples': hp.quniform('min_child_samples', 5, 50, 1),
    'n_estimators': hp.quniform('n_estimators', 100, 1000, 50),
    'subsample': hp.uniform('subsample', 0.6, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1),
}

def objective(params):
    # Преобразуем параметры, которые должны быть целыми числами
    params['num_leaves'] = int(params['num_leaves'])
    params['max_depth'] = int(params['max_depth']) if params['max_depth'] > 0 else -1
    params['min_child_samples'] = int(params['min_child_samples'])
    params['n_estimators'] = int(params['n_estimators'])
    
    # Удаляем параметры с некорректными значениями
    if params['max_depth'] == 0:
        params['max_depth'] = -1
    
    print(f"Текущие параметры: {params}")
    
    clf = lgb.LGBMClassifier(
        objective='binary',
        boosting_type='gbdt',
        class_weight='balanced',
        **params
    )
    
    # Используем кросс-валидацию для оценки модели
    try:
        score = cross_val_score(
            clf, X_resampled, y_resampled,
            scoring='roc_auc',
            cv=5,
            fit_params={'eval_metric': 'auc', 'verbose': -1}
        ).mean()
        loss = -score
        status = STATUS_OK
    except Exception as e:
        print(f"Ошибка при обучении модели: {e}")
        loss = np.inf
        status = STATUS_OK
    
    return {'loss': loss, 'status': status}

# Запуск оптимизации
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

print("Лучшие параметры:", best)

# Преобразуем параметры после оптимизации
best_params = {
    'num_leaves': int(best['num_leaves']),
    'max_depth': int(best['max_depth']) if best['max_depth'] > 0 else -1,
    'learning_rate': best['learning_rate'],
    'min_child_samples': int(best['min_child_samples']),
    'n_estimators': int(best['n_estimators']),
    'subsample': best['subsample'],
    'colsample_bytree': best['colsample_bytree'],
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'class_weight': 'balanced',
}




Текущие параметры: {'colsample_bytree': 0.9915384109973744, 'learning_rate': 0.04986544341201452, 'max_depth': 9, 'min_child_samples': 36, 'n_estimators': 650, 'num_leaves': 56, 'subsample': 0.8198126026525794}
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(



Ошибка при обучении модели:                           
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'verbose'

Текущие параметры: {'colsample_bytree': 0.7629628413533296, 'learning_rate': 0.0665887710764731, 'max_depth': 15, 'min_child_samples': 19, 'n_estimators': 1000, 'num_leaves': 57, 'subsample': 0.8915627563512318}
  2%|▏         | 1/50 [00:00<00:27,  1.76trial/s, best loss: inf]

C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(



Ошибка при обучении модели:                                      
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'verbose'

Текущие параметры: {'colsample_bytree': 0.8910670098786795, 'learning_rate': 0.08570169546579202, 'max_depth': 7, 'min_child_samples': 15, 'n_estimators': 700, 'num_leaves': 26, 'subsample': 0.925955631869724}
  4%|▍         | 2/50 [00:01<00:25,  1.91trial/s, best loss: inf]

...

Ошибка при обучении модели:                                       
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'verbose'

Текущие параметры: {'colsample_bytree': 0.8317084107164279, 'learning_rate': 0.027514224169851397, 'max_depth': 4, 'min_child_samples': 47, 'n_estimators': 750, 'num_leaves': 48, 'subsample': 0.6471951084338126}
 96%|█████████▌| 48/50 [00:25<00:01,  1.66trial/s, best loss: inf]

C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(



Ошибка при обучении модели:                                       
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'verbose'

Текущие параметры: {'colsample_bytree': 0.773930692568755, 'learning_rate': 0.03490199313126039, 'max_depth': 8, 'min_child_samples': 37, 'n_estimators': 400, 'num_leaves': 42, 'subsample': 0.7021716088944939}
 98%|█████████▊| 49/50 [00:26<00:00,  1.74trial/s, best loss: inf]

C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(



Ошибка при обучении модели:                                       
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\александр\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
TypeError: LGBMClassifier.fit() got an unexpected keyword argument 'verbose'

100%|██████████| 50/50 [00:26<00:00,  1.88trial/s, best loss: inf]
Лучшие параметры: {'colsample_bytree': 0.9915384109973744, 'learning_rate': 0.04986544341201452, 'max_depth': 9.0, 'min_child_samples': 36.0, 'n_estimators': 650.0, 'num_leaves': 56.0, 'subsample': 0.8198126026525794}


In [16]:
# Обучение модели с лучшими параметрами
lgb_clf_opt = lgb.LGBMClassifier(**best_params)
lgb_clf_opt.fit(
    X_resampled, y_resampled,
    eval_metric='auc'
)

# Предсказание и оценкаФ
y_pred = lgb_clf_opt.predict(X_test)
y_pred_proba = lgb_clf_opt.predict_proba(X_test)[:, 1]

print("recall:", recall_score(y_test, y_pred))
print("precision:", precision_score(y_test, y_pred))
print("roc-auc:", roc_auc_score(y_test, y_pred_proba))
print("confusion matrix:", confusion_matrix(y_test, y_pred))

[LightGBM] [Info] Number of positive: 37888, number of negative: 37888
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.115840 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 106732
[LightGBM] [Info] Number of data points in the train set: 75776, number of used features: 419
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
recall: 1.0
precision: 1.0
roc-auc: 1.0
confusion matrix: [[7571    0]
 [   0  176]]


## Модель Catboost

In [17]:
import catboost as ctb
from catboost import CatBoostClassifier

cat_clf = CatBoostClassifier(
    iterations=1000,              
    learning_rate=0.1,             
    depth=6,                       
    class_weights=class_weights,   
    eval_metric='AUC',             
    random_seed=42,               
    verbose=100)                   

cat_clf.fit(
    X_resampled, y_resampled,
    eval_set=(X_test, y_test),  
    early_stopping_rounds=50           
)

y_pred = cat_clf.predict(X_test)

y_pred_proba = cat_clf.predict_proba(X_test)[:, 1]

recall_ctb = recall_score(y_test, y_pred)
precision_ctb = precision_score(y_test, y_pred)
roc_auc_ctb = roc_auc_score(y_test, y_pred_proba)
conf_matrix_ctb = confusion_matrix(y_test, y_pred)

print("Recall:", recall_ctb)
print("Precision:", precision_ctb)
print("ROC-AUC:", roc_auc_ctb)
print("Confusion Matrix:\n", conf_matrix_ctb)

0:	test: 0.5129483	best: 0.5129483 (0)	total: 286ms	remaining: 4m 45s
100:	test: 0.8740672	best: 0.8740672 (100)	total: 7.06s	remaining: 1m 2s
200:	test: 0.9489537	best: 0.9489537 (200)	total: 14.5s	remaining: 57.6s
300:	test: 0.9785815	best: 0.9785815 (300)	total: 23.2s	remaining: 53.9s
400:	test: 0.9893516	best: 0.9893516 (400)	total: 30s	remaining: 44.9s
500:	test: 0.9923437	best: 0.9924195 (492)	total: 41.2s	remaining: 41.1s
600:	test: 0.9937073	best: 0.9937073 (596)	total: 48s	remaining: 31.9s
700:	test: 0.9946912	best: 0.9946927 (695)	total: 54.8s	remaining: 23.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9947969825
bestIteration = 721

Shrink model to first 722 iterations.
Recall: 1.0
Precision: 0.14789915966386555
ROC-AUC: 0.9947969825050131
Confusion Matrix:
 [[6557 1014]
 [   0  176]]


In [ ]:
в

## Модель RandomForest

In [9]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_jobs=-1, verbose=1, class_weight='balanced')
rf_clf.fit(X_resampled, y_resampled)

rf_y_pred = rf_clf.predict(X_test)
rf_y_pred_proba = rf_clf.predict_proba(X_test)[:, 1]
recall_rf = recall_score(y_test, rf_y_pred)
precision_rf = precision_score(y_test, rf_y_pred)
roc_auc_rf = roc_auc_score(y_test, rf_y_pred_proba)
conf_matrix_rf = confusion_matrix(y_test,rf_y_pred)

print("Recall:", recall_rf)
print("Precision:", precision_rf)
print("Roc-AUC:", roc_auc_rf)
print("Confusion Matrix:\n", conf_matrix_rf)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.1s finished


Recall: 1.0
Precision: 1.0
Roc-AUC: 1.0
Confusion Matrix:
 [[7571    0]
 [   0  176]]


[Parallel(n_jobs=6)]: Using backend ThreadingBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.0s
[Parallel(n_jobs=6)]: Done 100 out of 100 | elapsed:    0.0s finished


## Стак модель

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier  
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import recall_score, precision_score, roc_auc_score, confusion_matrix

class EstimatorWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator, name, **kwargs):
        self.name = name
        self.base_estimator = base_estimator
        self.base_estimator.set_params(**kwargs)
    
    def fit(self, X, y, **fit_params):
        print(f"Начинается обучение модели {self.name}")
        self.base_estimator.fit(X_resampled, y_resampled, **fit_params) ##вернуть х и у
        self.classes_ = self.base_estimator.classes_  
        print(f"Обучение модели {self.name} завершено")
        return self
    
    def predict(self, X):
        return self.base_estimator.predict(X)
    
    def predict_proba(self, X):
        return self.base_estimator.predict_proba(X)
    
    def get_params(self, deep=True):
        params = {'name': self.name, 'base_estimator': self.base_estimator}
        if deep:
            base_params = self.base_estimator.get_params(deep=deep)
            params.update(base_params)
        return params
    
    def set_params(self, **params):
        if 'name' in params:
            self.name = params.pop('name')
        if 'base_estimator' in params:
            self.base_estimator = params.pop('base_estimator')
        if params:
            self.base_estimator.set_params(**params)
        return self

lgb_clf_base = lgb.LGBMClassifier(**best_params)
cat_clf_base = CatBoostClassifier(
    class_weights=class_weights,
    random_seed=42,
    verbose=0
)
xg_clf_base = XGBClassifier()

lgb_clf_wrapped = EstimatorWrapper(lgb_clf_base, name='LightGBM', n_estimators=100, verbosity=2, n_jobs=-1)
cat_clf_wrapped = EstimatorWrapper(cat_clf_base, name='CatBoost',
                                   iterations=1000,
                                   learning_rate=0.1,
                                   depth=3,
                                   eval_metric='AUC',
                                   verbose=2)
xg_clf_wrapped = EstimatorWrapper(xg_clf_base, name='XGClassifier', verbosity=2, n_jobs=-1)

estimators = [
    ('lgb', lgb_clf_wrapped),
    ('cat', cat_clf_wrapped),
    ('xgb', xg_clf_wrapped)
]

meta_clf_base = XGBClassifier(
    use_label_encoder=False,  
    eval_metric='logloss',   
    random_state=42,
    n_estimators=100
)
meta_clf_wrapped = EstimatorWrapper(meta_clf_base, name='XGBoost')

stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_clf_wrapped,
    cv=5
)

stacking_clf.fit(X_resampled, y_resampled)

y_pred = stacking_clf.predict(X_test)
y_pred_proba = stacking_clf.predict_proba(X_test)[:, 1]

recall_stack = recall_score(y_test, y_pred)
precision_stack = precision_score(y_test, y_pred)
roc_auc_stack = roc_auc_score(y_test, y_pred_proba)
conf_matrix_stack = confusion_matrix(y_test, y_pred)

print("Recall:", recall_stack)
print("Precision:", precision_stack)
print("ROC-AUC:", roc_auc_stack)
print("Confusion Matrix:\n", conf_matrix_stack)

In [74]:
#заменить финального оценщика  и проверить RF
#найти лучшие параметры для других алгоритмов
#посмотреть со стандартизацией и PCA